In [44]:
import geopandas as gpd
import pandas as pd
import pysal
from pysal.lib import weights
from pysal.explore import esda
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [9]:
addresses = gpd.read_file("gis_data/Address_Points-shp/")

In [10]:
block_polys = gpd.read_file("output/result_street_dissolve.geojson")

In [18]:
owner_lots = gpd.read_file("gis_data/Common_Ownership_Lots-shp/")

In [14]:
block_polys.head(3)

,group,SQUARE,SSL,STNAME,SQUARE_PART,geometry
0,23,6239,6239S 0037,FORRESTER,1.0,"POLYGON ((-77.00704 38.82496, -77.00716 38.824..."
1,20,6239,6239S 0052,GALVESTON,1.0,"POLYGON ((-77.01010 38.82431, -77.01010 38.824..."
2,6,6239,6239S 0054,MARTIN LUTHER KING JR,1.0,"POLYGON ((-77.01155 38.82401, -77.01159 38.823..."


In [15]:
addresses.head(3)

,OBJECTID_1,OBJECTID,SITE_ADDRE,ADDRESS_ID,ROADWAYSEG,STATUS,SSL,TYPE_,ENTRANCETY,ADDRNUM,...,ACTIVE_RES,RES_TYPE,ACTIVE_R_1,WARD_2002,WARD_2012,ANC_2002,ANC_2012,SMD_2002,SMD_2012,geometry
0,832017,1087.0,27746,27746,11690.0,ACTIVE,0814 0018,ADDRESS,OFFICIAL,402.0,...,0.0,RESIDENTIAL,1.0,Ward 6,Ward 6,ANC 6C,ANC 6C,SMD 6C07,SMD 6C03,POINT (-77.00035 38.89220)
1,832018,3265.0,50808,50808,3073.0,ACTIVE,0838 0027,ADDRESS,OFFICIAL,219.0,...,0.0,RESIDENTIAL,1.0,Ward 6,Ward 6,ANC 6C,ANC 6C,SMD 6C07,SMD 6C03,POINT (-76.99925 38.89265)
2,832019,3266.0,51065,51065,3073.0,ACTIVE,0814 0818,ADDRESS,OFFICIAL,216.0,...,1.0,RESIDENTIAL,2.0,Ward 6,Ward 6,ANC 6C,ANC 6C,SMD 6C07,SMD 6C03,POINT (-76.99977 38.89256)


In [16]:
addresses.iloc[0]

OBJECTID_1                                              832017
OBJECTID                                                  1087
SITE_ADDRE                                               27746
ADDRESS_ID                                               27746
ROADWAYSEG                                               11690
STATUS                                                  ACTIVE
SSL                                               0814    0018
TYPE_                                                  ADDRESS
ENTRANCETY                                            OFFICIAL
ADDRNUM                                                    402
ADDRNUMSUF                                                None
STNAME                                            CONSTITUTION
STREET_TYP                                              AVENUE
QUADRANT                                                    NE
CITY                                                WASHINGTON
STATE                                                  

In [17]:
res_addresses = addresses[addresses.loc[:,'RES_TYPE']=='RESIDENTIAL'].copy()

In [19]:
owner_lots.head(3)

,OBJECTID,SQUAREPLYI,RESERVATIO,PARCELPLYI,TAXLOTSPLY,RECORDLOTS,ADDRESS_ID,ACCOUNT_ID,RECORDATIO,SQUARE,...,PY10COLL,PY10BAL,PY10CR,OWNNAME2,DEEDS,GIS_ID,CONDOLOT,SHAPEAREA,SHAPELEN,geometry
0,93122387,None,None,None,None,None,NaN,None,2016-07-13,4315,...,None,None,None,MAURA K GASWIRTH,None,897216.0,N,502.054648,101.049777,"POLYGON ((-76.96938 38.93033, -76.96953 38.930..."
1,93122388,None,None,None,None,None,NaN,None,1970-01-01,5612,...,None,None,None,None,None,1152981.0,N,299.952715,78.466363,"POLYGON ((-76.97918 38.86652, -76.97953 38.866..."
2,93122389,None,None,None,None,None,NaN,None,1970-01-01,1285,...,None,None,None,None,None,1142396.0,N,20000.108730,715.692934,"POLYGON ((-77.05439 38.91247, -77.05460 38.911..."


In [28]:
res_addresses_w_assessment = res_addresses[['SSL','FULLADDRES','geometry']].merge(owner_lots[['SSL','SALEPRICE','ASSESSMENT']], how='left', on=['SSL'])

In [29]:
res_addresses_w_assessment.head(5)

,SSL,FULLADDRES,geometry,SALEPRICE,ASSESSMENT
0,0814 0018,402 CONSTITUTION AVENUE NE,POINT (-77.00035 38.89220),981000.0,1114860.0
1,0838 0027,219 5TH STREET NE,POINT (-76.99925 38.89265),NaN,1125190.0
2,0814 0818,216 5TH STREET NE,POINT (-76.99977 38.89256),NaN,1156350.0
3,0838 0041,215 5TH STREET NE,POINT (-76.99923 38.89255),1635000.0,1426700.0
4,0866 0013,626 CONSTITUTION AVENUE NE,POINT (-76.99746 38.89219),NaN,1008850.0


In [41]:
len(res_addresses_w_assessment)

131803

In [39]:
addresses_w_price_poly = gpd.sjoin(res_addresses_w_assessment, block_polys[['group', 'SQUARE', 
                                                            'STNAME', 'SQUARE_PART', 'geometry']],op='within')

In [40]:
len(addresses_w_price_poly)

122315

In [42]:
addresses_w_price_poly.columns

Index(['SSL', 'FULLADDRES', 'geometry', 'SALEPRICE', 'ASSESSMENT',
       'index_right', 'group', 'SQUARE', 'STNAME', 'SQUARE_PART'],
      dtype='object')

In [47]:
len(block_polys)

17547

In [56]:
addresses_w_block_polys = addresses_w_price_poly.groupby(['SQUARE','SQUARE_PART','group']).agg({'ASSESSMENT':np.median}).reset_index()

In [57]:
block_polys_w_assessment = block_polys.merge(addresses_w_block_polys, how='left', on=['SQUARE','SQUARE_PART','group'])

In [58]:
block_polys_w_assessment.to_file("output/block_polys_w_assessment.geojson", driver="GeoJSON")